In [ ]:
!pip install gymnasium highway-env stable-baselines3[extra]

In [1]:
import gymnasium as gym
# import highway_env
# highway_env.register_highway_envs()

import time
import numpy as np
import pandas as pd
from collections import deque
from tqdm.notebook import tqdm
import itertools
import pickle
import random
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output

from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.monitor import Monitor

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from scipy.signal import convolve, gaussian
import tensorflow as tf

random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

In [2]:
def timeit(f):

    def timed(*args, **kw):

        ts = time.time()
        result = f(*args, **kw)
        te = time.time()

        print('func:%r took: %2.4f sec' % \
          (f.__name__, te-ts))
        return result

    return timed

action_names = ['left', 'idle', 'right', 'faster', 'slower']

In [3]:
default_config = {
    "lanes_count" : 4,
    "vehicles_count": 50,
    "duration": 40,
    "other_vehicles_type": "highway_env.vehicle.behavior.IDMVehicle",
    "initial_spacing": 2,
    "simulation_frequency": 15,
    "policy_frequency": 5,
    "collision_reward": -200,
    "right_lane_reward": 10,
    "high_speed_reward": 20,
    "lane_change_reward": 2,
    "reward_speed_range": [20, 30],
    "normalize_reward": False,
    "screen_width": 800,
    "screen_height": 600,
    "centering_position": [0.5, 0.5],
    "scaling": 5,
    "show_trajectories": True,
    "render_agent": True,
    "offscreen_rendering": False
}

In [4]:
class TimeToCollision:
    def __init__(self, 
                horizon=5,
                policy_frequency=5,
                simulation_frequency=15,
                high_speed_reward=10,
                right_lane_reward=10,
                lane_change_reward=-1,
                render_mode=None):

        self.config = default_config.copy()
        self.config["observation"] =  {
                "type": "TimeToCollision",
                "horizon": horizon}
        self.horizon = horizon

        self.config.update({
            "policy_frequency": policy_frequency,
            "simulation_frequency": simulation_frequency,
            "high_speed_reward": high_speed_reward,
            "right_lane_reward": right_lane_reward,
            "lane_change_reward": lane_change_reward
        })

        with gym.make("highway-fast-v0", config=self.config, render_mode=render_mode) as env:
            self.env = env
            self.obs_space_shape = env.observation_space.shape
            self.action_space_size = env.action_space.n

    def get_state(self):
        grid = self.env.get_wrapper_attr('vehicle').speed_index
        # return self.current_obs[grid]
        return self.current_obs

    def reset(self, seed='random'):
        if seed == 'random':
            seed = np.random.randint(100000)
        self.current_obs, info = self.env.reset(seed=seed)
        return self.get_state()

    def step(self, action):
        self.current_obs, reward, done, truncated, info = self.env.step(action)
        return self.get_state(), reward, done, truncated

    def close(self):
        self.env.close()

    def action_space_sample(self):
        return self.env.action_space.sample()

    def test_env(self, sleep_time=1):
        with gym.make("highway-v0", config=self.config, render_mode='human') as env:
            self.env = env
            obs = env.reset()
            for _ in range(1000):
                action = env.action_space.sample()
                self.current_obs, reward, done, truncated, info = env.step(action)
                print(self.get_state(env), reward)
                if done:
                    break
                time.sleep(sleep_time)

In [5]:
class TimeToCollisionVec:
    def __init__(self, 
                num_envs=1, 
                horizon=5,
                policy_frequency=5,
                simulation_frequency=15,
                high_speed_reward=10,
                right_lane_reward=10,
                lane_change_reward=-1,
                obs_type='speed-restricted',
                ):
        """
        Constructor for the TimeToCollisionVec class.
        Args:
            num_envs: Number of parallel environments to run.
            horizon: The number of timesteps to look ahead for the TimeToCollision observation.
            policy_frequency: The frequency at which the policy is evaluated.
            simulation_frequency: The frequency at which the simulation is run.
            high_speed_reward: The reward given for driving at high speed.
            right_lane_reward: The reward given for driving in the right lane.
            lane_change_reward: The reward given for changing lanes.
            obs_type: The type of observation to use. Choose from 'normal' or 'speed-restricted'
                'normal': The normal TimeToCollision observation (x3 different speeds the car could be at)
                'speed-restricted': Only one of the TimeToCollision observations is used (the speed the car is currently at) (less complex observation space)
        """
        
        self.config = default_config.copy()
        self.config["observation"] =  {
                "type": "TimeToCollision",
                "horizon": horizon}
        self.horizon = horizon

        self.config.update({
            "policy_frequency": policy_frequency,
            "simulation_frequency": simulation_frequency,
            "high_speed_reward": high_speed_reward,
            "right_lane_reward": right_lane_reward,
            "lane_change_reward": lane_change_reward
        })
        
        self.num_envs = num_envs
        self.obs_type = obs_type

        with gym.make("highway-fast-v0", config=self.config) as env:
            self.env = gym.vector.AsyncVectorEnv([lambda: env for _ in range(num_envs)])
            self.obs_space_shape = env.observation_space.shape if obs_type == 'normal' else env.observation_space.shape[1:]
            self.action_space_size = env.action_space.n

    def get_speeds(self):
        return [car.speed_index for car in self.env.get_attr('vehicle')]
    
    def get_state(self, obs):
        if self.obs_type == 'normal':
            return obs
        else:
            speed_indices = self.get_speeds()
            return np.array([obs[i][speed_indices[i]] for i in range(self.num_envs)])
    
    def reset(self, seeds=None):
        if seeds is None:
            seeds = [np.random.randint(100000) for _ in range(self.num_envs)]
        obs, info = self.env.reset(seed=seeds)
        return self.get_state(obs)
    
    def step(self, actions):
        obs, rewards, dones, truncates, info = self.env.step(actions)
        return self.get_state(obs), np.array(rewards), np.array(dones), np.array(truncates)
    
    def close(self):
        self.env.close()    

    def action_space_sample(self):  
        return self.env.action_space.sample()

In [6]:
def CNN(obs_space_shape, action_space_size):
    model = Sequential()
    model.add(Conv2D(64, (2, 2), input_shape=obs_space_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Dropout(0.2))

    model.add(Dense(action_space_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

def NN(obs_space_shape, action_space_size):
    # Make the multiplication in the obs space shape tuple 
    obs_size = (np.prod(obs_space_shape),)
    model = Sequential()
    model.add(Dense(128, input_shape=obs_size, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(action_space_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

In [13]:
class DQNAgent:
    def __init__(self, 
                 env,
                 model_type='CNN'):
        assert not (model_type == 'CNN' and env.obs_type == 'speed-restricted'), 'CNN model cannot be used with speed-restricted observation type'
        self.env = env
        self.model_type = model_type

        # Main model - what gets trained every step
        self.model = self.create_model()

        # Target model - what we predict every step
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())
        self.target_update_counter = 0

    def create_model(self):
        if self.model_type == 'CNN':
            return CNN(self.env.obs_space_shape, self.env.action_space_size)
        elif self.model_type == 'NN':
            return NN(self.env.obs_space_shape, self.env.action_space_size)
        
    def init(self, epsilon=1,
             min_eps=0.001,
             eps_decay=0.998,
             gamma=0.99,
             replay_memory=50_000,
             batch_size=64,
             update_weights_freq=5,
             min_replay_memory=1_000
             ):
        self.initialized = True
        self.epsilon, self.min_eps, self.eps_decay = epsilon, min_eps, eps_decay
        self.replay_memory = deque(maxlen=replay_memory)
        self.update_weights_freq = update_weights_freq
        self.min_replay_memory = min_replay_memory
        self.batch_size = batch_size
        self.gamma = gamma
        self.current_episode, self.step = 0, 0
        self.mean_rw_history, self.mean_steps_history, self.loss_hist, self.action_distribution_history = [], [], [], []

    def update_replay_memory(self, transition):
        self.replay_memory.extend(transition)

    def reshape_states(self, state):
        if self.model_type == 'CNN':
            return state
        elif self.model_type == 'NN':
            return state.reshape(-1, np.prod(state.shape[1:]))

    def get_qs(self, state):
        state_ = self.reshape_states(state)
        return self.model.predict(state_, verbose=0)

    @timeit
    def train_model(self, terminal_state):
        if len(self.replay_memory) < self.min_replay_memory:
            return

        minibatch = random.sample(self.replay_memory, self.batch_size)

        current_states = np.array([transition[0] for transition in minibatch])
        current_states_  = self.reshape_states(current_states)
        current_qs_list = self.model.predict(current_states_, verbose=0)                     # The model that gets trained every step

        new_current_states = np.array([transition[3] for transition in minibatch])
        new_current_states_ = self.reshape_states(new_current_states)
        future_qs_list = self.target_model.predict(new_current_states_, verbose=0)             # The model that doesn't get trained every step

        X,y = [], []

        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
            new_q = reward

            if not done:
                # In case of a non-terminal state, add future reward
                max_future_q = np.max(future_qs_list[index])
                new_q += self.gamma * max_future_q

            current_qs = current_qs_list[index]
            # Update the q value for the action taken
            current_qs[action] = new_q

            X.append(self.reshape_states(current_state))
            y.append(current_qs)

        history = self.model.fit(np.array(X), np.array(y), batch_size=self.batch_size, verbose=0, shuffle=False)

        if terminal_state:
            self.target_update_counter += 1
        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > self.update_weights_freq:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

        return history.history['loss'][0] 

    def epsilon_greedy(self,current_state):
        if np.random.random() > self.epsilon:
            action = np.argmax(self.get_qs(current_state), axis=1)
        else:
            # Generate self.env.num_envs random actions
            action = self.env.action_space_sample()
        return np.array(action)

    def decay_eps(self):
        if self.epsilon > self.min_eps:
            self.epsilon *= self.eps_decay
            self.epsilon = max(self.min_eps, self.epsilon)


    def train(self, episodes=10_000, episode_duration=200, evaluation_freq=25):
        if not self.initialized:
          print('Warning! Initializing with default parameters')
          self.init()

        start_time = time.time()
        for episode in tqdm(range(self.current_episode, episodes), unit='episode'):
            current_states = self.env.reset()
            self.current_episode += 1
            
            count = 0
            # Episode cycle
            while count < episode_duration:
                action = self.epsilon_greedy(current_states)
                new_states, rewards, done, truncate = self.env.step(action)
                done = done | truncate
                
                to_append = list(zip(current_states, action, rewards, new_states, done))
                self.update_replay_memory(to_append)
                # self.loss_hist.append(self.train_model(False))

                current_states = new_states
                self.step += 1
                count += 1

            loss = self.train_model(True)
            self.loss_hist.append(loss) if loss is not None else None

            if self.current_episode % evaluation_freq == 0:
                rewards_mean, steps_mean, action_distribution = self.evaluate_and_render(n_games=3, steps=200)
                self.mean_rw_history.append(rewards_mean)
                self.mean_steps_history.append(steps_mean)
                self.action_distribution_history.append(action_distribution)

                clear_output(wait=True)

                print(f"episode = {self.current_episode}, epsilon = {self.epsilon}, mean reward = {rewards_mean}, mean steps = {steps_mean}")
                print(f"total steps = {self.step*self.env.num_envs}, time_elapsed = {time.time() - start_time}, fps = {self.step*self.env.num_envs / (time.time() - start_time)}")
                
                plt.figure(figsize=[16, 10])
                plt.suptitle(f"Stats for episode {self.current_episode}")
                plt.subplot(2, 2, 1)
                plt.title("Mean reward per episode")
                plt.plot(self.mean_rw_history)
                plt.grid()

                plt.subplot(2, 2, 2)
                plt.title("Mean steps per episode")
                plt.plot(self.mean_steps_history)
                plt.grid()

                plt.subplot(2, 2, 3)
                plt.title("Action distribution for the current episode")
                plt.bar(action_names, action_distribution)
                plt.grid()

                plt.subplot(2, 2, 4)
                plt.title("Action distribution history for all episodes")
                action_distribution_history = np.array(self.action_distribution_history)
                for i in range(self.env.action_space_size):
                    plt.plot(action_distribution_history[:, i], label=f'{action_names[i]}')
                plt.legend()
                plt.grid()
                
                plt.show()

            self.decay_eps()

    def evaluate(self, agent, n_games=3, steps=200, verbose=True):
        sim_freq, pol_freq, horizon = self.env.config['simulation_frequency'], self.env.config['policy_frequency'], self.env.horizon
        ttc_vec_env = TimeToCollisionVec(num_envs=n_games, horizon=horizon, policy_frequency=pol_freq, simulation_frequency=sim_freq, render_mode=None)

        print('Evaluating...') if verbose else None
        total_reward, finished_games = 0, 0
        obs = ttc_vec_env.reset()
        actions_hist = []
        for _ in range(steps):
            actions = np.argmax(agent.get_qs(obs), axis=1)
            actions_hist.extend(actions)
            obs, rewards_, dones, _ = ttc_vec_env.step(actions)
            print(actions, rewards_)
            total_reward += rewards_.sum()
            finished_games += dones.sum()

        total_steps = steps * n_games
        average_reward = total_reward / finished_games if finished_games > 0 else 0
        average_steps = total_steps / finished_games if finished_games > 0 else steps
        # Calculate the action distribution
        actions_hist = np.array(actions_hist)
        action_distribution = np.array([np.sum(actions_hist == i) for i in range(ttc_vec_env.action_space_size)]) / len(actions_hist)
        ttc_vec_env.close()        
        return average_reward, average_steps, action_distribution


    def evaluate_and_render(self, n_games=3, steps=300, verbose=True):
        sim_freq, pol_freq, horizon = self.env.config['simulation_frequency'], self.env.config['policy_frequency'], self.env.horizon
        ttc_env = TimeToCollision(policy_frequency=pol_freq, simulation_frequency=sim_freq, horizon=horizon, render_mode='human')

        print('Evaluating...') if verbose else None
        rewards, steps_array, actions_hist = [], [], []
        for i in tqdm(range(n_games)):
            cum_reward = 0
            step, done = 0, False
            current_state = ttc_env.reset(seed=np.random.randint(100000+i))
            while not done and step < steps:
                # Get next action
                next_action = np.argmax(self.get_qs(current_state.reshape(1, *current_state.shape)))
                actions_hist.append(next_action)
                current_state, reward, done, truncated = ttc_env.step(next_action)
                done = done | truncated
                cum_reward += reward
                step += 1
            rewards.append(cum_reward)
            steps_array.append(step)
        actions_hist = np.array(actions_hist)
        action_distribution = np.array([np.sum(actions_hist == i) for i in range(ttc_env.action_space_size)]) / len(actions_hist)

        ttc_env.close()
        return np.mean(rewards), np.mean(steps_array), action_distribution

In [14]:
ttc_vec = TimeToCollisionVec(num_envs=10, 
                             horizon=5, 
                             policy_frequency=4, 
                             simulation_frequency=20,
                             obs_type='speed-restricted',
)
agent = DQNAgent(ttc_vec, model_type='NN')
agent.init(
    epsilon=1,
    min_eps=0.01,
    eps_decay=0.992,  #.998
    gamma=0.99,
    replay_memory=10_000,  #50_000
    batch_size=64,   # 64
    update_weights_freq=10,  #5
    min_replay_memory=1_000,
)

agent.train(episodes=200,
            episode_duration=50,
            evaluation_freq=15)

  0%|          | 0/200 [00:00<?, ?episode/s]

func:'train_model' took: 0.0000 sec


________________________________________

In [51]:
config = {
    "observation": {
        "type": "TimeToCollision"
    },
    "vehicles_count": 50,
    "duration": 120,
    "policy_frequency": 5,
    "simulation_frequency": 15
}
env = gym.make("highway-fast-v0")
env.configure(config)
env.reset()

model = DQN(
    "MlpPolicy",
    env,
    learning_rate=1e-3,
    buffer_size=50000,
    learning_starts=1000,
    batch_size=32,
    tau=1.0,
    gamma=0.99,
    train_freq=4,
    gradient_steps=1,
    target_update_interval=1000,
    exploration_fraction=0.1,
    exploration_final_eps=0.02,
    verbose=1
)

# Definieren Sie einen Checkpoint-Callback
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path='./models/', name_prefix='dqn_model')

# Trainieren Sie das Modell
model.learn(total_timesteps=100000, callback=checkpoint_callback)

# Speichern Sie das Modell
model.save("dqn_highway")

# Laden Sie das Modell
model = DQN.load("dqn_highway")


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 99.5     |
|    ep_rew_mean      | 80.7     |
|    exploration_rate | 0.961    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 79       |
|    time_elapsed     | 5        |
|    total_timesteps  | 398      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 94.4     |
|    ep_rew_mean      | 76       |
|    exploration_rate | 0.926    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 90       |
|    time_elapsed     | 8        |
|    total_timesteps  | 755      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 92.2     |
|    ep_rew_mean      | 75.5  

ValueError: You have passed a tuple to the predict() function instead of a Numpy array or a Dict. You are probably mixing Gym API with SB3 VecEnv API: `obs, info = env.reset()` (Gym) vs `obs = vec_env.reset()` (SB3 VecEnv). See related issue https://github.com/DLR-RM/stable-baselines3/issues/1694 and documentation for more information: https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api

In [56]:
env = gym.make("highway-fast-v0", render_mode="human", config=config)

while True:
  done = truncated = False
  obs, info = env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    env.render()

env.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyboardInterrupt: 